In [2]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file, to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

from module.conf import PROJECT_DIR

import tensorflow as tf
import numpy as np
import random
import sys
import io

In [3]:
strategy  = tf.distribute.get_strategy()
with strategy.scope():
    print(f"{tf.config.list_physical_devices('GPU') }")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
org_file = PROJECT_DIR + "/data/lstm/truyen_kieu_data.txt"
pre_file = PROJECT_DIR + "/data/lstm/truyen_kieu_data_pre.txt"

def pre_data() -> None:
    with open(file=org_file, mode="rt") as i_f:
        with open(file=pre_file, mode="wt") as o_f:
            for line in i_f:
                if "" == line.strip(): continue
                a_word = line.split()
                pre_word = ""
                for word in a_word:
                    tmp_line = "".join(filter(str.isalpha, word.strip()))
                    pre_word = " ".join((pre_word, tmp_line))
                    pass
                pre_word = pre_word.strip().lower()
                o_f.write(f"{pre_word}\n") 
                pass
            pass
        pass
    return

def load_data() -> tuple:
    rs: map = {}
    ls = []
    tmp_line = ""
    with open(file=pre_file, mode="rt") as i_f:
        count = 0
        for line in i_f:
            line = line.strip()
            if "" == line: continue
            if count % 2 == 0 and tmp_line!="":
                ls.append(tmp_line.strip())
                tmp_line = ""
                pass
            tmp_line += (" | " if tmp_line!="" else "") + line
            a_word = line.split()
            for word in a_word:
                if word not in rs:
                    rs[word] = 0
                    pass
                rs[word]+=1
                pass
            pass
            count+=1
        pass
    return rs, ls

pre_data()
bag_of_words, coupled_lines = load_data()
bag_of_words["|"] = len(coupled_lines)
# ix_to_word = [*bag_of_words.keys()]
keys = [*bag_of_words.keys()]
word_to_ix = { w:i for i,w in enumerate(keys) }
ix_to_word = { i:w for i,w in enumerate(keys) }
# coupled_lines
# values = [*bag_of_word.values()]
# sum(values)
# len(bag_of_words)
# ix_to_word
X_train = [[word_to_ix[word] for word in coupled_line.split()] for coupled_line in coupled_lines]

delta_line = 4
# y_train = [line[5] for line in X_train[delta_line:]]
y_train = [*X_train[delta_line:]]
y_train = [e[5] for e in y_train]
for c in range(delta_line):
    y_train.append(X_train[c][5])
    pass

total_word = len(bag_of_words)
max_input_len = 15
X_train = np.asarray(a=X_train, dtype=int)
y_train = np.asarray(y_train) #.reshape(X_train.shape[0], 1)
# print(y_train.shape)
y_train = to_categorical(y_train, num_classes=total_word, dtype=int)
y_train.shape

(1628, 2394)

In [ ]:
model: tf.keras.Model = tf.keras.Sequential(name="LSTM-RNN")
# input = tf.keras.layers.Input(shape=(15))
embedding_1 = tf.keras.layers.Embedding(input_dim=total_word, output_dim=1024, input_length=max_input_len)
bidrect_1 = tf.keras.layers.Bidirectional(\
    layer=tf.keras.layers.LSTM(units=160, return_sequences=True, go_backwards=False),\
    backward_layer=tf.keras.layers.LSTM(units=240, return_sequences=True, go_backwards=True))
bidrect_2 = tf.keras.layers.Bidirectional(layer=tf.keras.layers.LSTM(units=256, return_sequences=True))
dropout = tf.keras.layers.Dropout(rate=0.2)
lstm = tf.keras.layers.LSTM(units=128)
output_1 = tf.keras.layers.Dense(units=(total_word + 1)/2, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))
output_2 = tf.keras.layers.Dense(units=total_word, activation="softmax")

# model.add(input)
model.add(embedding_1)
# model.add(bidrect_1)
model.add(bidrect_2)
model.add(dropout)
model.add(lstm)
model.add(output_1)
model.add(output_2)

model.summary()

In [ ]:
tf.config.set_soft_device_placement(True) 
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# loss_fn = tf.keras.losses.sparse_categorical_crossentropy
loss_fn = tf.keras.losses.categorical_crossentropy
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),
              loss=loss_fn,
              metrics=["accuracy"])
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#               loss=loss_fn,
#               metrics=["accuracy"])

In [ ]:
# strategy  = tf.distribute.get_strategy()
with strategy.scope():
# with tf.device('/cpu:0'):
# with tf.device("/gpu:0"):
    model.fit(x=X_train, y=y_train, epochs=160, batch_size=64, verbose=1)
    pass

In [ ]:
num_of_couple_line = 2
